## Tutorial 5: Concave likelihood function of the RUM-MNL model

The log-likelihood function is a mathematical function that represents the log of the probability of observing the given data under a specific statistical model, as a function of the model parameters. It is used to estimate the parameters of the choice model that are most likely to have produced the observed data.<br><br>
Estimation of the parameters of choice models is usually done using Maximum Likelihood Estimation (MLE). MLE involves an iterative numerical process, typically using a Newton-Raphson, to recover the maximum likelihood estimates $\hat{\beta}$.
<br><br>
In case of the linear-additive 
However, the estimation algorithm may get stuck in **local** maxima. Local maxima are solutions where the function reaches a high value but not the highest possible. In contrast, a global maximum refers to the highest value of the function across its entire domain. In case, an optimisation algorithm gets stuck in a local solution, it does not recover the true maximum likelihood estimates. Such estimates are thus biased are should never be used for policy making. Hence, choice modellers should be aware of the possibility that optimisation algorithms can get stuck in local solutions. Unfortunately, there is no easy way to diagnose whether the optimiser found the global solution, or got stuck in a local one. 
<br><br>
**This tutorial will show why an optimiser can get stuck in local solutions and how it may be mitigated using different sets of starting values.**

In [1]:
# import libraries
import numpy as np
import plotly.graph_objects as go

For this tutorial we do not use choice data. Choice data are high-dimensional (usually they contain at least 4 attributes), which hampers a good visualisation of local versus global maxima of the loglikelihood function. Instead, we use a simpler/stylised likelihood function. This likelihood is a function of just two betas: $\beta_1$ and $\beta_2$ and has one local and a global maximum. 
<br><br>
Below we import this likelihood function. Also, we import a gradient_ascent function, which we will use later to visualise how an optimiser climbs the likelihood function. If you want to see, these scripts simply open the files in the editor. 

In [3]:
from likelihood_fcn import likelihood_function, gradient_ascent

#### Let's plot the likelihood function

In [4]:
# Create a 2D grid for plotting the likelihood function
beta1 = np.linspace(-2, 4, 100)
beta2 = np.linspace(-2, 4, 100)
B1, B2 = np.meshgrid(beta1, beta2)

# Calculate the likelihood for each point in the grid using the likelihood function
likelihood = likelihood_function(B1, B2)

# Create the surface plot using Plotly
# Plotly is a plotting library that makes interactive graphs
surface = go.Surface(z=likelihood, x=B1, y=B2, colorscale='Viridis', opacity=1, showscale=False)

# Plot the likelihood surface
fig = go.Figure(data=surface)

# Update the layout
fig.update_layout(
    title='Likelihood Function',
    scene=dict(
        xaxis_title='Beta1',
        yaxis_title='Beta2',
        zaxis_title='Likelihood',
        camera=dict(
        eye=dict(x=-1.8, y=-1.0, z=0.5))),
        height=800,
        width=800)

# Show the plot
fig.show()

`--> By rotating the plot, we can made a couple of observations regarding this likelihood function:`<br>
`1. The likelihood function is smooth (at least within the visible domain)`<br><br>
`2. The likelihood function has two maxima: (2.0, 1.2) and (-0.5, 2.0)`<br><br>
`3. The maximum at (-0.5, 2.0) is a local maximum (which we want to avoid)`<br><br>
`3. Far from the maxima (e.g. at the point (-2,-2)) the likelihood surface is almost flat`<br><br>

#### Add optimisation path to the likelihood surface plot given a `starting point`
Below we create a function which plots the likelihood surface (as above) as well as the **path** that an optimiser takes when climbing the likelihood function. The function `plot_likelihood_surface_with_paths` takes the following input arguments:
* the plotly surface of the likelihood function (which we created in the previous cell), and
* starting points: a list of tuples, where each tuple contains the starting point (beta1,beta2) for the optimiser.
<br>
 
Optionally, it takes parameters controlling the optimiser:
* stepsize: determines how far the optimiser moves in each iteration
* tolX: determines the tolerance for the gradient ascent. It stops the optimiser when the gradient is smaller than tolX 
* max_iter: sets the maximum number of iterations the optimiser is allowed to take.<br>

The gradient ascent function `gradient_ascent` climbs the log-likelihood function by iteratively moving in the direction of the steepest gradient. The gradient ascent algorithm stops when the Root Mean Squared Error of the gradients (in each direction) is smaller than tolX or when the maximum number of iterations is reached. The function returns the path that the optimiser took to climb the likelihood function.

In [5]:
def plot_likelihood_surface_with_paths(surface, start_points, stepsize = 0.3, tolX = 1e-6, max_iter = 2000):
    
    # Plot the surface of the likelihood function
    fig = go.Figure(data=surface)
    
    for start_point in start_points:
        
        # Perform optimisation
        points = gradient_ascent(start_point, stepsize, tolX=tolX, max_iter=max_iter)
        likelihood = likelihood_function(points[:, 0], points[:, 1])
        
        # Add optimisation path to the surface plot
        path = go.Scatter3d(
            x=points[:, 0], 
            y=points[:, 1], 
            z=likelihood, 
            mode='lines+markers', 
            line=dict(color='red', width=12), 
            marker=dict(size=6, color='red'),
            name='Path starting at ({}, {})'.format(start_point[0], start_point[1]))
        
        fig.add_trace(path)  # Add the path to the figure
        
    # Update the layout
    fig.update_layout(
        title='Likelihood Function with Gradient Ascent Paths',
        scene=dict(
            xaxis_title='Beta1',
            yaxis_title='Beta2',
            zaxis_title='Likelihood',
            camera=dict(
            eye=dict(x=-0.6, y=-2.0, z=0.2)
            )),
        width=1000,
        height=1000)  

    # Show the plot
    fig.show()

#### Starting the optimisation algorithm at (0,0)
Usually, when we estimate discrete choice models, we use 0 as the starting point for the optimiser. Let's see what happens when we start at (0,0) with this log-likelihood function.

In [6]:
# Plot the likelihood surface and the path that the optimiser takes when starting at (0,0). 
start_points =[(0,0)]
plot_likelihood_surface_with_paths(surface,start_points)

Converged in 79 iterations. RMSE of gradient at convergence: 8.82e-07


`--> Based on the plot above we can make a couple of observations:`<br><br>
`1. The optimiser gets stuck in the local maximum at (-0.5, 2.0).`<br><br>
`2. The optimiser iteratively climbs the likelihood function. It moves in the direction of the steepest gradient.`<br><br>
`3. The optimiser needed 79 iterations to get to the peak.`<br><br>
`4. The optimiser stopped because the Root Mean Squared Error of the gradient was smaller than TolX.`<br><br>
`5. Even though the optimiser convergence properly (i.e. based on the gradient tolerance), it did not reach the global maximum.`<br><br>
`6. For a researcher, it is impossible to know whether the optimiser found the global maximum or got stuck in a local one. After all, the algorithm properly converged (so he does not get a failed convergence message from the optimiser) and the he does not have a global view of the log-likelihood (as we have here).`<br><br>

#### Multiple starting points
One strategy to avoid local maxima is to try different starting points. Below we try 8 different starting points to see if the optimiser can reach the global maximum. We try the following 8 starting points:
* (0,0)
* (1,0)
* (0,1)
* (1,1)
* (2,2)
* (-2,-2)

In [7]:
# Plot the likelihood surface and the paths that the optimiser takes. 
start_points =[(0,0),(1,0),(0,1),(1,1),(-1,1),(-1,-1),(2,2),(-2,-2)]
plot_likelihood_surface_with_paths(surface,start_points)

Converged in 79 iterations. RMSE of gradient at convergence: 8.82e-07
Converged in 23 iterations. RMSE of gradient at convergence: 4.72e-07
Converged in 44 iterations. RMSE of gradient at convergence: 9.85e-07
Converged in 18 iterations. RMSE of gradient at convergence: 9.83e-07
Converged in 43 iterations. RMSE of gradient at convergence: 8.91e-07
Converged in 17 iterations. RMSE of gradient at convergence: 7.67e-07
Converged in 0 iterations. RMSE of gradient at convergence: 5.07e-08


`--> Based on the plot above a couple of observations can be made:`<br><br>
`1. Only 3 out of 8 starting points reach the global maximum`<br><br>
`2. 3 out of 8 starting points reach the local maximum` <br><br>
`3. 2 out of 8 do not reach a maximum at all`<br>
* `In particular, the path of starting point (-2,-2) "converged" immediately at iteration 0. This happens because the likelihood function is virtually flat at the initial starting point. The RMSE of gradient at the starting point is < TolX ;`<br>
* `The path of starting point (-1,-1) slowly climbed the likelihood function, but does not reach any maximum because it reached the maximum number of iterations. It is so slow because the surface is so flat around the starting point.`<br><br>


#### The concave likelihood function of the linear-additive RUM-MNL model

The likelihood function of the linear-additive Random Utility Model (RUM) is concave. This means that a well-specified linear-additive RUM-MNL model has only one maximum, which is the global maximum. This is why the optimiser always converges to the global maximum when estimating the parameters of the RUM-MNL model. However, the likelihood function of the RUM-MNL model is not always concave. 